# Homework: Agents
In this homework, we will learn more about function calling, and we will also explore MCP - model-context protocol.

In [1]:
# generate fake weather data
import random

known_weather_data = {
    'berlin': 20.0
}

def get_weather(city: str) -> float:
    city = city.strip().lower()

    if city in known_weather_data:
        return known_weather_data[city]

    return round(random.uniform(-5, 35), 1)

### Q1. Define function description
We want to use it as a tool for our agent, so we need to describe it

How should the description for this function look like? Fill in missing parts

In [2]:
get_weather_tool = {
    "type": "function",
    "name": "get_weather",
    "description": "Get weather in city",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "City name for which we want to get weather"
            }
        },
        "required": ["city"],
        "additionalProperties": False
    }
}

### Testing it (Optional)
If you have OpenAI API Key (or alternative provider), let's test it.\
A question could be "What's the weather like in Germany?"\
Experiment with different system prompts to have better answers from the system.\
You can use chat_assistant.py or implement everything yourself
```console
wget https://raw.githubusercontent.com/alexeygrigorev/rag-agents-workshop/refs/heads/main/chat_assistant.py
```

In [3]:
import chat_assistant

In [4]:
# add function search with search_description
tools = chat_assistant.Tools()
tools.add_tool(get_weather, get_weather_tool)

In [5]:
tools.get_tools()

[{'type': 'function',
  'name': 'get_weather',
  'description': 'Get weather in city',
  'parameters': {'type': 'object',
   'properties': {'city': {'type': 'string',
     'description': 'City name for which we want to get weather'}},
   'required': ['city'],
   'additionalProperties': False}}]

### Q2. Adding another tool

In [6]:
# add weather data to our database
def set_weather(city: str, temp: float) -> None:
    city = city.strip().lower()
    known_weather_data[city] = temp
    return 'OK'

In [7]:
set_weather_tool = {
    "type": "function",
    "name": "set_weather",
    "description": "Add weather in city in database",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "City name for which we want to add weather"
            },
            "temp": {
                "type": "float",
                "description": "Temperature in the city"
            }
        },
        "required": ["city", "temp"],
        "additionalProperties": False
    }
}

In [8]:
tools.add_tool(set_weather, set_weather_tool)

In [9]:
tools.get_tools()

[{'type': 'function',
  'name': 'get_weather',
  'description': 'Get weather in city',
  'parameters': {'type': 'object',
   'properties': {'city': {'type': 'string',
     'description': 'City name for which we want to get weather'}},
   'required': ['city'],
   'additionalProperties': False}},
 {'type': 'function',
  'name': 'set_weather',
  'description': 'Add weather in city in database',
  'parameters': {'type': 'object',
   'properties': {'city': {'type': 'string',
     'description': 'City name for which we want to add weather'},
    'temp': {'type': 'float', 'description': 'Temperature in the city'}},
   'required': ['city', 'temp'],
   'additionalProperties': False}}]

### Q3. Install FastMCP
MCP stands for Model-Context Protocol. It allows LLMs communicate with different tools (like Qdrant). It's function calling, but one step further:\
A tool can export a list of functions it has\
When we include the tool to our Agent, we just need to include the link to the MCP server

In [10]:
import fastmcp
print(fastmcp.__version__)

2.10.3


### Q4. Simple MCP Server

### Q5. Protocol
There are different ways to communicate with an MCP server. Ours is currently running using standart input/output, which means that the client write something to stdin and read the answer using stdout.

Our weather server is currently running.

This is how we start communitcating with it:

First, we send an initialization request -- this way, we register our client with the server:
```json
{"jsonrpc": "2.0", "id": 1, "method": "initialize", "params": {"protocolVersion": "2024-11-05", "capabilities": {"roots": {"listChanged": true}, "sampling": {}}, "clientInfo": {"name": "test-client", "version": "1.0.0"}}}
```
We should get back something like that, which is an aknowledgement of the request:
```json
{"jsonrpc":"2.0","id":1,"result":{"protocolVersion":"2024-11-05","capabilities":{"experimental":{},"prompts":{"listChanged":false},"resources":{"subscribe":false,"listChanged":false},"tools":{"listChanged":true}},"serverInfo":{"name":"Demo 🚀","version":"1.9.4"}}}
```
Next, we reply back, confirming the initialization:
```json
{"jsonrpc": "2.0", "method": "notifications/initialized"}
```
We don't expect to get anything in response\
Now we can ask for a list of available methods:
```json
{"jsonrpc": "2.0", "id": 2, "method": "tools/list"}
```
Let's ask the temperature in Berlin:
```json
{"jsonrpc": "2.0", "id": 3, "method": "tools/call", "params": {"name": "get_weather", "arguments": {"city": "berlin"}}}
```
What did you get in response?
```json
{"jsonrpc":"2.0","id":3,"result":{"content":[{"type":"text","text":"20.0"}],"structuredContent":{"result":20.0},"isError":false}}
```

### Q6. Client
We typically don't interact with the server by copy-pasting commands in the terminal.\
In practice, we use an MCP Client. Let's implement it.\
FastMCP also supports MCP clients:

Use the client to get the list of available tools of our script. How does the result look like?

If you're running this code in Jupyter, you need to pass an instance of MCP server to the Client:

In [18]:
import weather_server
from fastmcp import Client

async def main():
    async with Client(weather_server.mcp) as mcp_client:
        tools = await mcp_client.list_tools()
        print("Available tools:")
        print(tools)

await main()

Available tools:
[Tool(name='get_weather', title=None, description='Retrieves the temperature for a specified city.\n\nParameters:\n    city (str): The name of the city for which to retrieve weather data.\n\nReturns:\n    float: The temperature associated with the city.', inputSchema={'properties': {'city': {'title': 'City', 'type': 'string'}}, 'required': ['city'], 'type': 'object'}, outputSchema={'properties': {'result': {'title': 'Result', 'type': 'number'}}, 'required': ['result'], 'title': '_WrappedResult', 'type': 'object', 'x-fastmcp-wrap-result': True}, annotations=None, meta=None), Tool(name='set_weather', title=None, description="Sets the temperature for a specified city.\n\nParameters:\n    city (str): The name of the city for which to set the weather data.\n    temp (float): The temperature to associate with the city.\n\nReturns:\n    str: A confirmation string 'OK' indicating successful update.", inputSchema={'properties': {'city': {'title': 'City', 'type': 'string'}, 'tem

If you run it in a script, you will need to use asyncio:

### Using tools from the MCP server (optional)
FastMCP uses asyncio for client-server communication. In our case, the code we wrote previously in the module (chat_assistant.py) is not asyncio-friendly, so it will require a lot of adjustments to run it.\
Which is why we asked Claude to implement a simple non-async MCP client (see mcp_client.py) that can only do this:\
* List tools
* Invoke the specified tool
Note: this is not a production-ready MCP Client! Use it only for learning purposes.

In [19]:
import mcp_client

our_mcp_client = mcp_client.MCPClient(["python", "weather_server.py"])

our_mcp_client.start_server()
our_mcp_client.initialize()
our_mcp_client.initialized()

Started server with command: python weather_server.py
Sending initialize request...
Initialize response: {'protocolVersion': '2024-11-05', 'capabilities': {'experimental': {}, 'prompts': {'listChanged': False}, 'resources': {'subscribe': False, 'listChanged': False}, 'tools': {'listChanged': True}}, 'serverInfo': {'name': 'Demo 🚀', 'version': '1.10.1'}}
Sending initialized notification...
Handshake completed successfully


In [20]:
our_mcp_client.get_tools()
our_mcp_client.call_tool('get_weather', {'city': 'Berlin'})

Retrieving available tools...
Available tools: ['get_weather', 'set_weather']
Calling tool 'get_weather' with arguments: {'city': 'Berlin'}


{'content': [{'type': 'text', 'text': '20.0'}],
 'structuredContent': {'result': 20.0},
 'isError': False}

In order to include it in our existing application, we need a wrapper class:

In [21]:
import json

class MCPTools:
    def __init__(self, mcp_client):
        self.mcp_client = mcp_client
        self.tools = None
    
    def get_tools(self):
        if self.tools is None:
            mcp_tools = self.mcp_client.get_tools()
            self.tools = convert_tools_list(mcp_tools)
        return self.tools

    def function_call(self, tool_call_response):
        function_name = tool_call_response.name
        arguments = json.loads(tool_call_response.arguments)

        result = self.mcp_client.call_tool(function_name, arguments)

        return {
            "type": "function_call_output",
            "call_id": tool_call_response.call_id,
            "output": json.dumps(result, indent=2),
        }

Let's use it:

In [22]:
our_mcp_client = mcp_client.MCPClient(["python", "weather_server.py"])

our_mcp_client.start_server()
our_mcp_client.initialize()
our_mcp_client.initialized()

mcp_tools = mcp_client.MCPTools(mcp_client=our_mcp_client)


developer_prompt = """
You help users find out the weather in their cities. 
If they didn't specify a city, ask them. Make sure we always use a city.
""".strip()

chat_interface = chat_assistant.ChatInterface()

chat = chat_assistant.ChatAssistant(
    tools=mcp_tools,
    developer_prompt=developer_prompt,
    chat_interface=chat_interface,
    client=client
)

chat.run()

Started server with command: python weather_server.py
Sending initialize request...
Initialize response: {'protocolVersion': '2024-11-05', 'capabilities': {'experimental': {}, 'prompts': {'listChanged': False}, 'resources': {'subscribe': False, 'listChanged': False}, 'tools': {'listChanged': True}}, 'serverInfo': {'name': 'Demo 🚀', 'version': '1.10.1'}}
Sending initialized notification...
Handshake completed successfully


NameError: name 'client' is not defined